# REASON Radargram Demo

This notebook demonstrates how to use the `reason_pds_review` library to load and visualize REASON partially processed data.

## Setup

First, import the necessary libraries:

In [ ]:
import sys

import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from reason_pds_review import load_ppdp, apply_stacking

# Configure matplotlib for better output
%matplotlib inline
plt.rcParams['figure.dpi'] = 100

## Configuration

Set up paths and processing parameters:

In [ ]:
# Path to partially processed data
data_dir = "../../urn-nasa-pds-clipper.rea.partiallyprocessed/DATA/000MGA/2025060T1736"

# Stacking configuration (incoherent averaging)
# Reduces noise and number of pulses for easier visualization
STACKING = {
    'HF': 10,          # 10x stacking for HF channel
    'VHF_POSX': 20,    # 20x stacking for VHF channels
    'VHF_FULL': 20,
    'VHF_NEGX': 20
}

# Channel ordering for subplot layout (left to right, top to bottom)
CHANNEL_ORDER = ['HF', 'VHF_POSX', 'VHF_FULL', 'VHF_NEGX']

## Load Data

Use the `load_ppdp()` function to load all channels from the data directory:

In [ ]:
print(f"Loading data from {data_dir}...")
tree = load_ppdp(data_dir)

print("\nData loaded successfully!")
print(f"Available channels: {list(tree.children.keys())}")

## Explore the Data Structure

The data is organized in a hierarchical xarray DataTree:

In [ ]:
# Display the full tree structure
tree

### Examine HF Channel Data

Let's look at the HF science data in detail:

In [ ]:
# Access HF science dataset
hf_science = tree['HF/science'].ds
print(hf_science)

In [ ]:
# Display metadata
print("HF Channel Metadata:")
print(f"  Frequency: {hf_science.attrs['frequency']}")
print(f"  Sample Rate: {hf_science.attrs['sample_rate_hz']/1e6:.1f} MHz")
print(f"  Description: {hf_science.attrs['description']}")
print(f"  Data Shape: {hf_science.sizes}")
print(f"  Fast Time Range: 0 to {float(hf_science.coords['fast_time'].max()):.2f} μs")

### Examine Engineering Data

Check the chirp parameters from the engineering data:

In [ ]:
# Access HF engineering dataset
hf_eng = tree['HF/engineering'].ds

hf_eng

In [ ]:
# keys_to_plot = ['Chirp_start_frequency', 'Chirp_end_frequency', 'Doppler_frequency',
#                 'Dwell_ID', 'RX_window_length_ticks']
keys_to_plot = ['HW_RX_opening_ticks', 'TX_start_ticks', 'Chirp_length_ticks',
                'Raw_active_mode_length', 'RX_window_length_ticks', 'RX_window_time']
fig, axs = plt.subplots(len(keys_to_plot), 1, figsize=(10, 8), constrained_layout=True)
for ax, key in zip(axs, keys_to_plot):
    hf_eng[key].plot(ax=ax)
    ax.set_title(key)
plt.show()

In [ ]:
#tmp = hf_eng['HW_RX_opening_ticks'] - hf_eng['TX_start_ticks'] + hf_eng['Chirp_length_ticks']
tmp = hf_eng['Raw_active_mode_length'] / hf_eng['RX_window_length_ticks']
tmp.plot()

### Examine MED Data

Check the chirp parameters from the MED data:

In [ ]:
hf_med = tree['HF/med']

hf_med

In [ ]:
hf_med['Receive_time_offset'].plot()